In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model

In [2]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [3]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

### Register metric

### Create the config

#### Set data, model, trainer and metric

In [5]:
from federatedscope.core.configs.config import global_cfg, CN

cfg = global_cfg.clone()
cfg.data.type = 'cikm_cup'
cfg.data.root = 'data'
cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.1307], 'std': [0.3081]}]]
cfg.model.type = 'gin'
#cfg.model.out_channels = 10
cfg.model.task='graph'
cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'encoder', 'clf']
cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 32
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.01
cfg.params.csd_importance=100
cfg.params.eps=1e-5
cfg.params.p=0.

#### configure other options

In [6]:
cfg.use_gpu = True
cfg.best_res_update_round_wise_key = "test_loss"

cfg.federate.mode = 'standalone'
cfg.federate.method = 'Laplacian'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 1
cfg.federate.client_num = 13
cfg.early_stop.patience = 50
#cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.00001
cfg.grad.grad_clip = 5.0

cfg.criterion.type = 'CrossEntropyLoss'
cfg.seed = 123
cfg.eval.best_res_update_round_wise_key = "test_loss"

In [7]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup_per_client_own.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [9]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [10]:
from federatedscope.contrib.workers.laplacian_client import LaplacianClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-09-06 17:40:20,340 (utils:129)INFO: the current machine is at 127.0.1.1
2022-09-06 17:40:20,340 (utils:131)INFO: the current dir is /home/michael/Master thesis/Code/FederatedScope_thesis
2022-09-06 17:40:20,341 (utils:132)INFO: the output dir is exp/Laplacian_gin_on_cikm_cup_lr0.1_lstep1_/sub_exp_20220906174020
2022-09-06 17:40:20,341 (cikm_cup:58)INFO: Loading CIKMCUP data from /home/michael/Master thesis/Code/FederatedScope_thesis/data/CIKM22Competition.
2022-09-06 17:40:20,342 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #1.
2022-09-06 17:40:20,575 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #2.
2022-09-06 17:40:20,592 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #3.
2022-09-06 17:40:20,918 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #4.
2022-09-06 17:40:20,937 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #5.
2022-09-06 17:40:20,957 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #6.
2022-09-06 17:40:21,216 (cikm_cup:68)INFO: Loading CIKMCUP d

In [ ]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

In [ ]:
len(data[1]['train'].dataset)

In [11]:
import torch
data = [[1, 2],[3, 4]]
data2 = [[10, 20],[30, 40]]
x_data = torch.tensor(data)
x_data2 = torch.tensor(data2)
print(x_data)
print(x_data2)

tensor([[1, 2],
        [3, 4]])
tensor([[10, 20],
        [30, 40]])


In [11]:
data

{1: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7fb4b59e3b20>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36ce15850>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36c09ad30>,
  'num_label': 0},
 2: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36bfa8a30>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36bf3d850>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36bfdff40>,
  'num_label': 0},
 3: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36b9262b0>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36bcf16d0>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36bf5ca90>,
  'num_label': 0},
 4: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36b7c0d90>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36b7cd7f0>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7fb36b82adc0>,
  'num_label': 0},


In [15]:
print(type(x_data2))

<class 'torch.Tensor'>


In [14]:
x_data2[0]

tensor([1, 2])

Hey, a short update:
I implemented the method from the paper "A Bayesian Federated Learning Framework with
Online Laplace Approximation" in FederatedScope.
I played around with it but I couldn't really get good results.